# Image Classification with pretrained CNNs

In this notebook we want to use a CNN with our prepared image dataset. So far we used the nice and clean Fashion MNIST dataset to create a model. 

This time it's your turn to preprocess the raw, collected data!


## Obtaining and Preprocessing the Data

We already provide two image collections on the server for you.

- In the folder [*/data/image_cats_dogs*](/tree/data/image_cats_dogs) are 100 cat and 100 dog images. They come from the Kaggle-Challenge [*Dogs vs. Cats*](https://www.kaggle.com/c/dogs-vs-cats)
- In the folder [*/data/image_lion_puma*](/tree/data/image_lion_puma) are some images of lions and pumas, intentionally badly chosen to show the limits of learning methods.  
- If you trust yourself to create your own dataset, you can do it by the same procedure. For example you could develop a George Clooney vs. Brad Pitt classifier. Therefore you only have to upload some images of them on this Jupyter server.

*** If you haven't already done it, you should execute the [exercise 1.2](/notebooks/exercises/1.2_ImageProcessing_Exercise.ipynb) for image processing the cat vs. dog dataset (or modify it to use the other dataset). After that we can start with this notebook and compare a simple CNN with a pretrained CNN. ***

In [ ]:
# the usual imports
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
from IPython.display import Image, display

In [ ]:
# This is our current path
print("current path: {}".format(os.getcwd()))

# let's save the home path: this value is for the container environment. 
# You can update the path according to the outputof the command above
home = "/home/jovyan/"

# a mapping for human readable labels
labels = { 0 : "cat", 1 : "dog"}

# make sure you have the data ready: this should show some images
folders = {home + "temp/image_cats_dogs/train/dog/"}
for folder in folders:
    for i, file in zip(range(3),os.listdir(folder)):
        display(Image(filename=(folder + file)))

## Define and train the model 

Now we can start with the actual neural network itself. Therefore we need to import the deep learning libraries and create some constants for the dataset: (code is provided, just run it)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Flatten, Dense, Dropout
from keras import backend as K
import keras

img_size = 250

num_train = 160
num_test = 40

train_data_dir = '/home/jovyan/temp/image_cats_dogs/train'
test_data_dir  = '/home/jovyan/temp/image_cats_dogs/test'

### Define the model

We define a sequential network, which passes the data from input to the direction of output. 

The components of the Convolutional Network will be the same, as you've already seen in the exercises before.

***Hints:***
- the required elements are `Conv2D`, `Activation`, `MaxPooling`, `Flatten` and `Dense`
- we have to compile the network definition with `Sequential.compile` and define the loss function as well as an optimizer 
- `Sequential.summary()` summarizes the layer and the number of degrees of freedom( = learning parameter)
- start with two Convolutional Layer and reduce the dimension with Pooling 

In [ ]:
# Depending on the backend the order of dimension parameter is different
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_size, img_size)
else:
    input_shape = (img_size, img_size, 3)

model = Sequential()
Conv2D?
model.add(Conv2D(32, (7, 7), input_shape=input_shape))
#The model architecture could look as followed: 
#Conv2D, Activation, MaxPooling2D |Conv2D, Activation, MaxPooling2D | Flatten, Dense, Activation, Dense, Activation
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(7, 7)))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(1))
# in this case we have a binary classifier -> sigmoid with one output node
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

### Define the training process

Now we have to combine the data with the network. This is the same process like in the Fashion MNIST example. 

In [ ]:
epochs = 12 # we want to process each image twelve times, otherwise our augmentation would not make much sense. See below.
batch_size = 20

The model can be directly fed with data (`fit`) or can get the data from a generator (`fit_generator`). As we want to increase our data on-the-fly with augmentation, we define this generator and derive another generator form this previous generator, which reads the data in the training folder.

This results in a pipeline in which we can ask for new images infinite times. Each time we ask for an image, the generator takes the next file in the training data folder, applies a random augmentation and supplies it as an input to the training loop.

***Hints:***
- The interger color values (0-255) should be normalized to the range from 0 to 1 with the `rescale` method. 

In [ ]:
# augmentation 
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=5,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary')

Finally we can start the training process. Therefore we have to implement the `fit_generator` method with the training data generator and the desired number of epochs. After that we wait until the training is finished. Waiting is an important part of deep learning. Always have a good paper ready to read ;-)

In [ ]:
history = model.fit_generator(
    train_generator,
    epochs=epochs)

# Because we don't want to wait again, we store the resulting model in HDF5 format
model.save_weights('MyCatDogConv.h5')

In [ ]:
from matplotlib import pyplot as plt

# Progress of accuracy
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

## Evaluation 

Now we want to test the performance of our model for the test data. There is a method analogous to `fit_generator`, called `evaluate_generator`. 

This provides us with the usual evaluation metrics. If we want the actual prediction results, too, e.g.  to compute the confusion matrix, we can use the method `model.predict_generator`.

***Hint***: Don't forget to `rescale` to the same value range (0-1) and set `shuffle` to `false` !

In [ ]:

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_size, img_size),
    batch_size=num_test,
    shuffle=False,
    class_mode='binary')

scores = model.evaluate_generator(test_generator)
predictions = model.predict_generator(test_generator)

print("loss: {}, Acc: {} ".format(scores[0], scores[1]))

The predictions are not binary value (because we used a sigmoid activation), but a value between 0 and 1. If we want assign a class, we employ a threshold (usually 0.5, but depending on the application one class can be more important.)

Because you already done confusion matrices earlier, here is the complete code:

In [ ]:
import numpy as np
import sklearn.metrics as skm

# we know that the test data starts with all cats and then all dogs
truth = np.concatenate([np.zeros(20),np.ones(20)])

# threshold = 0.5 --> round
preds = np.around(predictions)

# the common thing:
print("Accuracy: {}".format(skm.accuracy_score(truth,preds)))
cm = skm.confusion_matrix(truth,preds)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm, cmap=plt.cm.gray)
fig.colorbar(cax)
ax.set_xticklabels(['']+list(labels.values()))
ax.set_yticklabels(['']+list(labels.values()))
plt.show()

## Transfer learning

With the limited datasets (cats vs. dogs and lions vs. pumas) the values probably wouldn't get any better. The task of training the computer to "see" from scratch is too difficult with so few examples. An alternative is taking a pretrained network, which is already trained on many image features and it can recognize them. We only take the last part of this pretrained CNN (Dense layers after convolutional layer, also called *Bottleneck*) and retrain them with our data according to our needs.

The idea is, that the model has already learned how to see and understand image parts from working with many, many images. We only want to learn which image parts and features make up a dog or a cat.

Therefore we download the current and already trained CNN (in our case: VGG16) without the upper layer and calculate the CNN output for our data without having trained it: 

In [ ]:
from keras import applications
import numpy as np

# no augmentation
datagen = ImageDataGenerator(rescale=1. / 255)

# Pretrained CNN without the upper layers 
vgg16 = applications.VGG16(include_top=False, weights='imagenet', input_shape=(250,250,3))

# Process the training data and save the ouput in a file 
generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
bottleneck_features_train = vgg16.predict_generator(
        generator, num_train // batch_size)
np.save(open("bottleneck_features_train.npy", "wb"), bottleneck_features_train)

# The same for test data
generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
bottleneck_features_test = vgg16.predict_generator(
        generator, num_test // batch_size)
np.save(open("bottleneck_features_test.npy", "wb"),
            bottleneck_features_test)

Then we model a very simple neural net and call it 'topModel'. This model shall learn what features make up a cat or dog.

This time we directly integrate the evaluation into the training process by defining the test data, too (`validation_data`)

In [ ]:
# load the CNN output again
train_data = np.load(open("bottleneck_features_train.npy", "rb"))
train_labels = np.array([0] * (num_train // 2) + [1] * (num_train // 2))
test_data = np.load(open("bottleneck_features_test.npy", "rb"))
test_labels = np.array([0] * (num_test // 2) + [1] * (num_test // 2))

# define a small simple CNN
topModel = Sequential()
topModel.add(Flatten(input_shape=train_data.shape[1:]))
topModel.add(Dense(256, activation='relu'))
topModel.add(Dropout(0.5))
topModel.add(Dense(1, activation='sigmoid'))

topModel.compile(optimizer='adam',
            loss='binary_crossentropy', metrics=['accuracy'])

# and train it
topModel.fit(train_data, train_labels,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(test_data, test_labels))

topModel.save_weights("top_model.h5")

And now we have great results, without augmentation and very few training examples!

(to be fair, cats and dogs were already part of the pretraining, so this was an easy tasks. Finding guns and drugs would maybe be more difficult)